<br><br><br><br><br><h1 style="font-size:4em;color:#2467C0">Welcome to Week 3</h1><br><br><br>


<p style="line-height:31px;">This document provides a running example of completing the Week 3 assignment : </p>
<ul class="simple">
<li style="line-height:31px;">You can then copy paste the commands below in pySpark. </li>
<li style="line-height:31px;">To open pySpark, type pyspark and hit enter</li>
<br>
Finally, make sure that your working directory contains the data file(s) (.csv) for the fastest support. <br><br> We recommend workng in your home directory (type cd ~ in your terminal). Please run any scripts using your terminal for proper settings
<br>

<p style="font-family: Arial; font-size:1.5em;color:#2462C0; font-style:bold">
<br><br><br>
To run these commands in Cloudera VM: first run the setup script: setupWeek3.sh</p>

In [19]:
from pyspark.sql import SQLContext
from pyspark.ml.clustering import KMeans
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StandardScaler

from notebooks import utils
%matplotlib inline

ImportError: No module named 'notebooks'

<br><br>

<p style="font-family: Arial; font-size:1.5em;color:#2462C0; font-style:bold">

Import Data
</p>

 

<br><br>
<div style="color:black;font-family: Arial; font-size:1.1em;line-height:65%">

<p style="line-height:31px;">First let us read the contents of the file w3_clustering.csv. The following commands read in the CSV file <br> in a table format. <br><br>


If you want to run this command on your machine: Note that you must change the path in the following <br>
command to 'w3_clustering.csv' to point to the location on your local machine.
</p>

</div>

<br><br><br><br>

In [2]:
sqlContext = SQLContext(sc)
data = sqlContext.read.load('file:///home/cloudera/Downloads/capstone/final-dataset.csv', 
                          format='com.databricks.spark.csv', 
                          header='true',inferSchema='true')

<br><br>

 
<p style="font-family: Arial; font-size:1.5em;color:#2462C0; font-style:bold">

Let us display the number of lines in the data:

</p>

<br><br>

In [3]:
data.count()

1388

<br><br>

 
<p style="font-family: Arial; font-size:1.5em;color:#2462C0; font-style:bold">

What does the data contain ?
</p>

 
<br><br>

In [4]:
data

DataFrame[count_gameclicks: int, adcount: int, buycount: int, revenue: int]

<br><br>

 
<p style="font-family: Arial; font-size:1.5em;color:#2462C0; font-style:bold">

Statistics about the data:
</p>

 
<br><br>

In [5]:
data.describe().toPandas().transpose()

,0,1,2,3,4
summary,count,mean,stddev,min,max
count_gameclicks,1388,145.63616714697406,119.04485769820992,6,982
adcount,1388,6.356628242074928,3.1009049785511875,1,17
buycount,1388,1.6880403458213256,0.901456342413272,1,6
revenue,1388,12.048991354466859,12.856239825114008,1,100


<br><br>

 
<p style="font-family: Arial; font-size:1.5em;color:#2462C0; font-style:bold">

Drop all rows with NULL or NaN values

</p>

 
<br><br>

In [6]:
data = data.na.drop()

<br><br>

 
<p style="font-family: Arial; font-size:1.5em;color:#2462C0; font-style:bold">

Let us look at the column names:

</p>

 
 
<br><br>

In [7]:
data.columns

['count_gameclicks', 'adcount', 'buycount', 'revenue']

<br><br>

 
<p style="font-family: Arial; font-size:1.5em;color:#2462C0; font-style:bold">

Use VectorAssembler to gather all the features:

</p>

 
<br><br>

In [11]:
featuresUsed = ['count_gameclicks', 'adcount', 'buycount', 'revenue']
assembler = VectorAssembler(inputCols=featuresUsed, outputCol="features_unscaled")
assembled = assembler.transform(data)

<br><br>

 
<p style="font-family: Arial; font-size:1.5em;color:#2462C0; font-style:bold">

Scale the features using StandardScaler:

</p>

 
<br><br>

In [12]:
scaler = StandardScaler(inputCol="features_unscaled", outputCol="features", withStd=True, withMean=True)
scalerModel = scaler.fit(assembled)
scaledData = scalerModel.transform(assembled)

<br><br>

 
<p style="font-family: Arial; font-size:1.5em;color:#2462C0; font-style:bold">

Select the features column make the data persist:

</p>

 
<br><br>

In [13]:
scaledData = scaledData.select("features")
scaledData.persist()

DataFrame[features: vector]

<br><br>

 
<p style="font-family: Arial; font-size:1.5em;color:#2462C0; font-style:bold">

We can now perform KMeans clustering to generate 2 clusters:
</p>

 
<br><br>

In [16]:
kmeans = KMeans(k=3, seed=1)
model = kmeans.fit(scaledData)
transformed = model.transform(scaledData)

<br><br>

 
<p style="font-family: Arial; font-size:1.5em;color:#2462C0; font-style:bold">

Print the center of these two clusters:

</p>

 
<br><br>

In [17]:
centers = model.clusterCenters()
centers

[array([ 2.24759699, -0.31287725, -0.28572406, -0.3399284 ]),
 array([-0.16449386,  0.26350707,  1.24961212,  1.08841518]),
 array([-0.34052252, -0.0529435 , -0.46914192, -0.39204793])]

<br><br>

 
<p style="font-family: Arial; font-size:1.5em;color:#2462C0; font-style:bold">

Analyze center of these two clusters:

</p>

 
<br><br>


<div style="color:black;font-family: Arial; font-size:1.1em;line-height:65%">

<p style="line-height:31px;">Each array denotes the center for a cluster:<br><br>
One Cluster is centered at   ... array([ 0.84174521,  0.51884657])<br>
Other Cluster is centered at   ... array([-0.79780796, -0.49176392])</p>

<br><br>

<p style="line-height:31px;"> First number (field1) in each array refers to scaled verson of the number of ad-clicks and the second number (field2) is the scaled version of the revenue per user.

Compare the 1st number of each cluster to see how differently users in each cluster behave when it comes to clicking ads.

Compare the 2nd number of each cluster to see how differently users in each cluster behave when it comes to buying stuff. 

</p><br><br>

<p style="line-height:31px;">In one cluster, in general, players click on ads much more often and spend more money on in-app purchases. Assuming that Eglence Inc. gets paid for showing ads and for hosting in-app purchase items, we can use this information to increase game's revenue by increasing the prices for ads we show to the frequent-clickers, and charge higher fees for hosting the in-app purchase items shown to the higher revenue generating buyers.</p>

<br><br>
<p style="line-height:31px;"> <b> Note: </b>  This analysis requires you to compare the cluster centers and find any ‘significant’ differences in the corresponding feature values of the  centers. The answer to this question will depend on the features you have chosen. <br><br> Some features help distinguish the clusters remarkably while others may not tell you much. At this point, if you don’t find clear distinguishing patterns, perhaps re-running the clustering model with different numbers of clusters and revising the features you picked would be a good idea. </p>

</div>


In [18]:
P = utils.pd_centers(featuresUsed, centers)

NameError: name 'utils' is not defined